In [2]:
from time import sleep
from tqdm import tqdm

In [3]:
import numpy as np
import pandas as pd
import requests

**Мега функция**

Отдает нам район по адресу в формате "улица, город, страна"

In [ ]:
def to_district(address: str, city: str, district_cache) -> str: # фун - я для опр. района по адресу. Используем API OSM
    """
    :address: тут мы поулчаем адрес, уже сформированный правильным образом для передачи в OSM
    :city: передаем город, нужен для cache, чтобы обращаться конкретно к кешу города
    :district_cache: очев. кэш
    """
    url = "https://nominatim.openstreetmap.org/search"
    if city not in district_cache: # для нового города создаем кэш
        district_cache[city] = {}
        
    if address in district_cache[city]: # проверка наличия адреса в кэше города
        return district_cache[city][address]
    
    params = {
      "q": address,
      "format": "json",
      "addressdetails": 1, # input
      "limit": 1 # output
    }
    
    headers = {
        "User-Agent": "RealEstateDistrictFetcher/1.0 (klvalopatin@edu.hse.ru; https://example.com)",
        "Accept-Language": "ru"
    }

    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        result = f"Ошибка запроса: {e}"
        district_cache[city][address] = result
        return result

    sleep(0.1)

    if not data:
        result = "Адрес не найден"
        district_cache[city][address] = result
        return result

    address_info = data[0].get('address', {})
    district = (
        address_info.get("city_district") or    
        address_info.get("district") or          
        address_info.get("borough") or           
        address_info.get("municipality") or       
        address_info.get("suburb") or           
        address_info.get("neighbourhood") or     
        address_info.get("quarter") or          
        address_info.get("city_block") or         
        address_info.get("state") or               
        "Район не найден"
    )
    district_cache[city][address] = district
    return district or "Район не найден"

**Сохраняем и создаем общий файл со всеми городами**

In [ ]:
estate_msk = pd.read_csv('../Web-parsing/data_estates/estate_msk.csv')
estate_spb = pd.read_csv('../Web-parsing/data_estates/estate_spb.csv')
estate_novosibirsk = pd.read_csv('../Web-parsing/data_estates/estate_novosibirsk.csv')
estate_nNovg = pd.read_csv('../Web-parsing/data_estates/estate_nizhnij-novgorod.csv')
estate_krasnoyarsk = pd.read_csv('../Web-parsing/data_estates/estate_krasnoyarsk.csv')
estate_chelyabinsk = pd.read_csv('../Web-parsing/data_estates/estate_chelyabinsk.csv')
estate_krasnodar = pd.read_csv('../Web-parsing/data_estates/estate_krasnodar-1.csv')


In [ ]:
estate_cities = pd.concat([
    estate_msk,
    estate_spb,
    estate_novosibirsk,
    estate_krasnodar,
    estate_chelyabinsk,
    estate_nNovg,
    estate_krasnoyarsk
])
estate_cities = estate_cities.reset_index(drop=True)

In [60]:
estate_cities['OSM_address'] = estate_cities['full_address'] + ', ' + estate_cities['city'] + ', Россия'

In [9]:
tqdm.pandas()


In [11]:
district_cache = {}



In [ ]:
estate_cities["district"] = estate_cities.progress_apply( # мега функция для всех 13к строк (район создаем в табличку)
    lambda x: to_district(x['full_address'], x['city'], district_cache),
    axis=1
)

100%|██████████| 13866/13866 [2:37:37<00:00,  1.47it/s]  


In [ ]:
estate_cities.to_csv("../Web-parsing/data_estates/estate_cities.csv", index=False) # сохр


In [37]:
estate_cities = estate_cities.drop("Unnamed: 0", axis=1).reset_index(drop=True)

In [ ]:
def to_normAdress(address: str):
    return address.split(',')[0]



Тут проблемка была, что много адресов было не найдено (сложная регулярка, так еще и нерабочая была, теперь крутая и простая функция). Доделаем 
для оставшихся адресов

In [14]:
mask = estate_cities["district"] == "Адрес не найден"

estate_cities.loc[mask, "district"] = estate_cities.loc[mask].progress_apply(
    lambda x: to_district(to_normAdress(x["full_address"]), x["city"], district_cache),
    axis=1
)


100%|██████████| 2424/2424 [10:21<00:00,  3.90it/s] 


In [ ]:
estate_cities.to_csv("../Web-parsing/data_estates/estate_cities.csv") # перезаписали